In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


## Prequisite
Non stationarity means with statistical properties, like mean, variance, and autocorrelation change over time.
<br>Types of Non-Stationarity
1. **Deterministic Trends**: Trend that can me modeled with deterministic function, ex: linear or polynomial trend.
2. **Stochastic Trends**: Trend that follow a random walk or a unit root process, where the series has a presistent, random component that causes the series to drift over time,
3. **Seasonal Non-Stationarity**: Regular patterns or cycles that repeat over a fixed period, such as monthly sales data showing seasonal effects.

## Identifying Non-Stationarity
1. Visual.
2. ADF(Statistical): Test the presence of unit root.
3. Kwiatkowski-Phillips-Schmidt-Shin (KPSS)(Statistical): test against the alternative of a unit root.
4. Phillips-Perron (PP) test: testing uni roots.

## Stationarization
This is more like a follow up from my Dickey-Fuller Test what is used to check if a time series data is stationary. <br>
After we check the time series data, if its found to be non-stationary then we can turn it into stationary using these process. <br>

- Financial institution and corporations, as well as individual investor and researcher usually use time series data like exchange rates, GDP, inflation, and other macroeconomic indicator in economic forecast, stock market analysis.

## 1. Differencing
- Subtracting the previous observation from the current observation. This is known as first differenceing. Higher order differencing can be used if necessary.